Environment

In [1]:
import os
import sys
sys.path.append("/home/zanardi/Codes/ML/RONEK/ronek/")

from ronek import env
env.set(
  device="cpu",
  device_idx=0,
  nb_threads=8,
  floatx="float64"
)

Import libraries

In [2]:
import copy
import numpy as np
import scipy as sp
import ronek.postproc.plotting as pltt

from ronek import const
from ronek import utils
from ronek.systems import TASystem
from ronek.roms import CoarseGraining
from silx.io.dictdump import h5todict

Define inputs
> System

In [3]:
T = 1e4
ic = {
  "cold": [5e2, 1e3, 5e-2],
  "hot":  [3e3, 1e3, 5e-2]
}
tgrid = {
  "lim": [1e-9, 1e-3],
  "num": 500
}
teval = {
  "cold": [1e-8, 1e-7, 7e-7, 3e-6, 1e-4],
  "hot":  [1e-7, 1e-6, 4e-6, 2e-5, 1e-3]
}

> Paths

In [4]:
path_to_dtb = "/home/zanardi/Codes/ML/RONEK/run/RVC_O3/database/"
path_to_data = "/home/zanardi/Codes/ML/RONEK/run/RVC_O3/test/data/"

Initialize isothermal master equation model

In [5]:
system = TASystem(
  T=T,
  rates=path_to_dtb + "/kinetics.hdf5",
  species={
    k: path_to_dtb + f"/species/{k}.json" for k in ("atom", "molecule")
  },
  use_einsum=False,
  use_factorial=False
)

In [6]:
cg = CoarseGraining(
  T=T,
  molecule=path_to_dtb + "/species/molecule.json"
)
e = cg.molecule.lev["e"]

In [ ]:
# Thermo
# ===================================
def build_interp(species, T, phi):
  if isinstance(phi, np.ndarray):
    phi = [phi]
  e = np.stack([species.lev["e"] @ phij for phij in phi], axis=0).T
  # Temperature to internal energy
  species.T_to_e = []
  for eg in e:
    species.T_to_e.append(
      (T.min(), T.max(), sp.interpolate.interp1d(T, eg, kind="linear", axis=0))
    )
  # Internal energy to temperature
  species.e_to_T = []
  for eg in e:
    species.e_to_T.append(
      (eg.min(), eg.max(), sp.interpolate.interp1d(eg, T, kind="linear", axis=0))
    )

def interpolate(species, T=None, e=None):
  if (T is not None):
    return species._interpolate(T, species.T_to_e)
  else:
    return species._interpolate(e, species.e_to_T)

def _interpolate(species, x, funs):
  y = [fun[2](np.clip(x[i], fun[0], fun[1]) ) for (i, fun) in enumerate(funs)]
  return np.array(y).reshape(-1)


In [7]:
def build_probmat(nb_bins):
  mapping = cg.get_mapping(nb_bins)
  return cg.construct_probmat(mapping)

def build_phi(T, P):
  cg.molecule.update(T)
  phi = P * cg.molecule.q.reshape(-1,1)
  Q = P.T @ cg.molecule.q
  phi /= Q.reshape(1,-1)
  return phi

def build_psi(P):
  return P, P*e.reshape(-1,1)

In [8]:
nb_bins = 5
Tint = np.geomspace(3e2, 5e5, 500)

ops = {
  "ed": [[],[]],
  "r": []
}
P = build_probmat(nb_bins)
psis = build_psi(P)
phis = []
for Ti in Tint:
  phi = build_phi(Ti, P)
  phis.append(phi)
  for j, psi in enumerate(psis):
    system.update_rom_ops(phi, psi, biortho=False)
    ops["ed"][j].append(system.rom_ops["ed"])
    if (len(ops["r"]) < 2):
      ops["r"].append(system.rom_ops["r"])
    if ("m_ratio" not in ops):
      ops["m_ratio"] = system.rom_ops["m_ratio"]
ops["ed"] = [np.stack(op, axis=0) for op in ops["ed"]]

utils.map_nested_dict(ops, np.shape)

{'ed': [(500, 5, 5), (500, 5, 5)], 'r': [(5,), (5,)], 'm_ratio': (1, 5)}

In [9]:
def build_interp_ops(Tint, ops):
  funs = copy.deepcopy(ops)
  for m in range(2):
    funs_m = []
    for op in np.transpose(ops["ed"][m], axes=(2,0,1)):
      funs_m.append(
        sp.interpolate.interp1d(Tint, op, kind="linear", axis=0)
      )
    funs["ed"][m] = funs_m
  return funs

def apply_interp_ops(T, funs):
  ops = copy.deepcopy(funs)
  for m in range(2):
    ops_m = []
    for i, fun in enumerate(funs["ed"][m]):
      ops_m.append(fun(T[i]))
    ops["ed"][m] = np.stack(ops_m, axis=-1)
  return ops

In [10]:
ops_interp = build_interp_ops(Tint, ops)
cg.molecule.build_interp(Tint, phis)

In [11]:
def fun(t, x, interp):
  # Extract variables
  x = x * const.UNA
  n_a, (n_m, e_m) = x[:1], np.split(x[1:], 2)
  # Interpolate temperatures
  T_m = cg.molecule.interpolate(e=e_m/n_m)
  print(T_m)
  # Interpolate operators
  ops = apply_interp_ops(T_m, interp)
  # Compose source terms
  # > Molecule mass
  w_n_m = ops["ed"][0] @ n_m * n_a \
        + ops["r"][0] * n_a**3
  # > Molecule energy
  w_e_m = ops["ed"][1] @ e_m * n_a \
        + ops["r"][1] * n_a**3
  # > Atom
  w_n_a = - ops["m_ratio"] @ w_n_m
  # Compose full source term
  f = np.concatenate([w_n_a, w_n_m, w_e_m], axis=0)
  return f / const.UNA

In [12]:
def encode(n, psis):
  return np.concatenate([n @ psi for psi in psis])

# def decode(z, P):
#   if (z.shape[1] != 2*nb_bins):
#     z = z.T
#   n_m, e_m = np.split(z, 2, axis=-1)
#   T_m = np.vstack([cg.molecule.interpolate(e=e_m[i]/n_m[i]) for i in range(len(z))])
#   for i, Ti in enumerate(T_m.T):
#     Ti = Ti.reshape(-1,1)
    
    
#     T_m @ P.T


#   return np.concatenate([n @ psi for psi in psis])

In [13]:
filename = path_to_data + "/case_hot.p"
icase = utils.load_case(filename=filename)
n_fom, t, n0 = [icase[k] for k in ("n", "t", "n0")]

In [14]:
z_m = encode(n0[1:], psis)
n_m, e_m = np.split(z_m, 2)
y0 = np.concatenate([n0[:1], n_m, e_m])
y0

array([9.05371315e+20, 1.65654615e+22, 6.27311768e+20, 9.19214744e+18,
       8.91247676e+16, 4.37862476e+14, 1.50206288e+03, 2.13674580e+02,
       5.67881456e+00, 7.84447377e-02, 4.91086410e-04])

In [15]:
y = system.solve(
  t=system.get_tgrid(1e-9,1e-3,500),
  y0=y0,
  fun=fun,
  jac=None,
  ops=ops_interp,
  rtol=1e-10
)

[4000.0891869  4000.05295357 4000.04824249 4000.06235384 4000.06899741]
[4000.08918849 4000.05277542 4000.04577977 4000.01981562 3999.65317565]
[4000.08918849 4000.05277542 4000.04577977 4000.01981562 3999.65317568]
[4000.08919008 4000.05259727 4000.04331704 3999.97727743 3999.23735615]
[4000.08919008 4000.05259727 4000.04331704 3999.97727744 3999.23735619]
[4000.09561524 3999.33252683 3990.09119341 3831.70628099 2579.52322537]
[4000.09561524 3999.33252683 3990.09119289 3831.70643595 2579.53693685]
[4000.09010628 3999.94991506 3998.62387439 3975.59097367 3766.80024061]
[4000.09010628 3999.94991506 3998.62387439 3975.59097466 3766.80028254]
[4000.09102249 3999.84723408 3997.20453785 3951.37580969 3546.17257335]
[4000.09102249 3999.84723408 3997.20453785 3951.37581083 3546.1726569 ]
[4000.09193869 3999.74455435 3995.78530739 3927.17657326 3336.11137145]
[4000.09193869 3999.74455435 3995.78530739 3927.17657295 3336.11147965]
[4000.09171771 3999.76932028 3996.1276119  3933.00950794 3385.86

capi_return is NULL
Call-back cb_f_in_lsoda__user__routines failed.


KeyboardInterrupt: 

In [ ]:
np.sort(np.random.choice(200,5,replace=False))

In [ ]:
i = np.random.choice(200)
Ti = np.full(5, Tint[i])
opsp = apply_interp_ops(Ti, ops_interp)

true = ops["ed"][1][i]
pred = opsp["ed"][1]
err = 100*np.abs(true - pred) / np.abs(true)
err.max()

In [ ]:
egp = np.exp(sp.interpolate.interp1d(Tint, np.log(e_g), kind="cubic", axis=0)(Tint))
err = 100*np.abs(egp - e_g) / np.abs(e_g)
err.max()

In [ ]:
eg_int = sp.interpolate.interp1d(Tint, np.log(e_g), kind="cubic", axis=0)
Tint_p = np.random.normal(1,0.2,size=nb_bins)*5000
egp = np.exp(eg_int(Tint_p))
egp

In [ ]:
egp = np.exp(sp.interpolate.interp1d(Tint, np.log(e_g), kind="cubic", axis=0)(Tint))
err = 100*np.abs(egp - e_g) / np.abs(e_g)
err.max()

In [ ]:
k_edp = np.sign(k_ed) * np.exp(sp.interpolate.interp1d(Tint, np.log(np.abs(k_ed)), kind="cubic", axis=0)(Tint))
err = 100*np.abs(k_edp - k_ed) / np.abs(k_ed)
err.max()

In [ ]:


  # Isothermal master equation model
  # -----------------------------------
  path_to_dtb = inputs["paths"]["dtb"]
  system = utils.get_class(
    modules=[sys_mod],
    name=inputs["system"]["name"]
  )(
    rates=path_to_dtb + "/kinetics.hdf5",
    species={
      k: path_to_dtb + f"/species/{k}.json" for k in ("atom", "molecule")
    },
    **inputs["system"]["kwargs"]
  )

  # Testing
  # -----------------------------------
  # Initialization
  # ---------------
  # Path to saving
  path_to_saving = inputs["paths"]["saving"] + "/error/" + inputs["eval_err_on"]
  os.makedirs(path_to_saving, exist_ok=True)
  # Time grid
  t = utils.load_case(path=inputs["data"]["path"], index=0, key="t")

  # ROM models
  bt_bases = h5todict(inputs["paths"]["bases"])
  bt_bases = [bt_bases[k] for k in ("phi", "psi")]
  cg_model = CoarseGraining(
    T=system.T,
    molecule=path_to_dtb+"/species/molecule.json"
  )

> Molecule's levels

In [ ]:
gi = model.species["molecule"].lev["g"]
ei = model.species["molecule"].lev['e'] / const.eV_to_J

Balanced POD

In [ ]:
path = paths["data"]+"/figs/bases/"
os.makedirs(path, exist_ok=True)

In [ ]:
bases = h5todict(paths["data"]+"/bases.hdf5")
s, phi, psi = [bases[k] for k in ("s", "phi", "psi")]

In [ ]:
# Cumulative energy
cs = 1.0 / np.sum(s**2)
cs *= np.cumsum(s**2)
# Number of principal components
eps = 1e-6
rom_dim = np.where(cs > 1-eps)[0][0]+1
rom_dim

In [ ]:
pltt.cum_energy(
  cs[:rom_dims[-1]],
  figname=path + "/cum_en",
  save=True,
  show=False
)

In [ ]:
for i in range(rom_dims[-1]):
  nb = str(i+1)
  for (name, basis) in (("phi", phi), ("psi", psi)):
    pltt.dist_2d(
      x=ei,
      y=basis[:,i],
      labels=[r"$\epsilon_i$ [eV]", r"$\%s_{%s}$" % (name, nb)],
      scales=["linear", "linear"],
      markersize=1,
      figname=path + f"/{name}_{nb.zfill(2)}",
      save=True,
      show=False
    )

FOM and ROM solutions

In [ ]:
t = get_tgrid(tgrid["lim"], tgrid["num"])
for (name, (T, p, Xa)) in ic.items():
  y0 = get_y0(model, T, p, Xa)
  # Solving
  print(f"> Solving FOM for '{name}' test case ...")
  yfom = solve_fom(model, t, y0)
  for r in rom_dims:
    print(f"  > Solving ROM for '{name}' test case with {r} dimensions ...")
    yrom = solve_rom(model, t, y0, phi, psi, r, abs=False)
    # Postprocessing
    print(f"  > Postprocessing FOM and ROM solutions ...")
    path = paths["data"] + f"/figs/sol/{name}_r{r}/"
    os.makedirs(path, exist_ok=True)
    # > Moments
    plot_moments(path, t, yfom[1], yrom[1], ei.reshape(-1,1), max_mom=2)
    # > Distribution static
    plot_dist(path, teval[name], t, yfom[1], yrom[1], ei, gi, markersize=1)
    # > Distribution dynamic
    animate_dist(path, t, yfom[1], yrom[1], ei, gi, markersize=1)